In [ ]:
import pickle


def goodtext(txt: str):
    newtxt = ''
    for i in txt:
        if i == '\n':
            i = ' '
        if (ord(i) > 1039 and ord(i) < 1104) or \
           (ord(i) in [32, 33, 44, 45, 46, 59, 63, 1025, 1105]):
            newtxt += i
    newtxt = newtxt.lower()
    return(newtxt)


def wordsplitter(newtxt: str):
    liste = []
    string = ''
    for i in newtxt:
        if i not in [' ', ',', ';', '.', '!', '?']:
            string += i
        elif (i != ' '):
            if string != '':
                liste.append(string)
            string = ''
            liste.append(i)
        elif string != '':
            liste.append(string)
            string = ''
    return(liste)


def dicofdic(liste: list, ngramm: int):
    N = ngramm - 1
    bigdic = {}
    startlist = []
    for i in range(len(liste)-N):
        if liste[i+N] in ['.', '?', '!']:
            startlist.append(tuple(liste[i+1 : i+N+1]))
        if tuple(liste[i : i+N]) not in bigdic.keys():
            bigdic[tuple(liste[i : i+N])] = {liste[i+N]:1}
        elif liste[i+N] in bigdic[tuple(liste[i : i+N])].keys():
            bigdic[tuple(liste[i : i+N])][liste[i+N]] += 1
        else:
            bigdic[tuple(liste[i : i+N])][liste[i+N]] = 1
    return(bigdic, startlist)


txt = ''
for i in range(9):
    j = str(i+1) + '.txt'
    f = open(j,'r')
    txt += f.read()
    f.close()
    
dic,start = dicofdic(wordsplitter(goodtext(txt)), 3)

with open('1.pickle', 'wb') as f:
    pickle.dump(dic, f)
with open('2.pickle', 'wb') as f:
    pickle.dump(start, f)

In [ ]:
import random
import pickle


def makingtxt(dic: dict, startlist: list, Lengt: int):
    finaltxt = ''
    Nkey = random.choices(startlist)[0]
    titlecount = 1
    while len(txt) < Lengt:
        choic = []
        weig = []
        for i in dic[Nkey].keys():
            choic.append(i)
            weig.append(dic[Nkey][i])
        new = random.choices(choic, weights=weig)[0]
        if new in ['.', '?', '!']:
            finaltxt = finaltxt[:-1] + new + ' '
            titlecount = 1
        elif new in [',', ';']:
            finaltxt = finaltxt[:-1] + new + ' '
            titlecount = 0
        else:
            if titlecount == 1:
                new = new[0].upper() + new[1:]
            finaltxt = finaltxt + new + ' '
            new = new.lower()
            titlecount = 0
        Nkey = list(Nkey[1:])
        Nkey.append(new)
        Nkey = tuple(Nkey)


with open('1.pickle', 'rb') as f:
    dic = pickle.load(f)
with open('2.pickle', 'rb') as f:
    start = pickle.load(f)
    
w.open('3000.txt', 'w')
w.write(makingtxt(dic, start, 3000))
w.close()

**Описание "генератора текста"**. Выше представлено два кода, ниже - пример генерации (3000 символов).<br>
*(в дальнейшем примем Ключ как ключ-кортеж, Значение - значение Ключа, ключ - ключ Значения, значение - значение ключа)*

**Первый код:**<br>
_Отвечает за редактирование исходных файлов, создание "словаря из словарей" для заданной N-граммы, т.н. "стартового списка" и их запись в файлы ля дальнейшего использования._<br>
**Подробно:**<br>
<font color=blue>[4]</font>.  *goodtext(txt)* записывает в *newtxt* русские буквы, пробелы и всевозможные знаки препинания, после чего преобразует newtxt в нижний регистр.<br>
<font color=blue>[15]</font>.  *wordsplitter(txt)* записывает слова и знаки препинания из *newtxt* в список *liste* (слова накапливаются в string)<br>
<font color=blue>[32]</font>.  *dicofdic(liste, ngramm)* из списка *liste* для заданной $N$-граммы создаёт "словарь из словарей" *bigdic* (словарь, где Ключи - кортежи длины $N$, а Значения - словари с ключами - всевозможными словами, идущими после данного Ключа и значениями - количеством встречающихся таких слов) и "стартовый список" *startlist* (его элементы - Ключи из "словаря из словарей" оканчивающиеся знаком конца предложения. Он нужен для того, чтобы можно было в будущем сгенерировать случайное первое слово).<br>
<font color=blue>[49]</font>.  В *txt* записывается строка из 9 текстовых файла - я взял оригинальный переведенный "Гарри Поттер" - все 8 частей и известный фанфик "Гарри Поттер - методы рационального мышления". В результате *txt* содержит порядка 10 млн слов. Далее генерируются вышеописанные словарь и список для триграмм и отправляются в соответствующие файлы с помощью *pikle*.<br>
    
**Второй код:**<br>
*Отвечает за генерацию текста путем использования _random_ с весами.*<br>
**Подробно:**<br>
<font color=blue>[5]</font>.  *makingtxt(dic,startlist,Lengt)* генерирует текст. Чтобы это сделать, берем из "словаря из словарей" случайный стартовый Ключ _Nkey_, для него нужно выбрать один из ключей Значения с вероятностью, соответствующей частоте значений в тексте; - Это можно сделать руками (брать сумму значений в данном Значении (пусть это $S$), после чего использовать _random($S$)_ и, разбив $S$, брать нужную часть), но проще и компактнее использовать _random(choices,weights=..)_; Далее сдвигаем Ключ-кортеж и  проделываем ту же процедуру пока не достигнем нужного количества символов *Lengt*. (<font color=blue>[8]</font>. *titlecount* отвечает за заглавную букву в начале предложений)<br>

**P.S.1** _Легко заметить, что при использовании биграмм получается совсем несвязный текст, а при $N$>3 из исходного текста происхоит слишком много копирования (поэтому оптимально использовать триграммы, см. ниже)._<br>
**P.S.2** *Можно было осуществить деление на абзацы путем добавления символа $\n\n$ при генерации словаря, либо при выводе после _random_ предложений, но, так как текст получается довольно несвязный, в этом мало необходимости*. <br>
**P.S.3** *Качество текста можно улучшить, если, например, сделать имена с заглавной буквы. К сожалению, найти все имена - (как оказывается) сложная и нерациональная задача, поэтому оставим как есть.* <br>
**P.S.4** *К сожалению, при заданном Lengt нельзя окончить текст знаком окончания предложения (вероятность этого достаточно мала...). Ситуацию можно было бы подправить несколькими способами: урезав текст до последнего знака окончания предложения; наоборот, "дождавшись" его после Lengt во время вывода; или же увеличивать вероятность наступления завершения по мере приближения к Lengt (но это совсем нетривиально).*

Некоторое время он мало путешествовал. Человек, который выглядел ещё более хмурым, когда сириус исчез в соседней комнате. На площадке никого не видела ничего подобного, я начинаю думать, твой отец он сделал что? Возопил мистер гойл попробовал использовать на мне, что, даже у пэнси паркинсон снова согнулась пополам от смеха, переспросила она и вернулась к телескопам, изучали ночное небо и думал о своём поступке. Не хочу быть собой. Послышался звук шагов одного человека. Нацисты попытались переправить уже очищенный дейтерий, погрузив его на допросы. Он говорит, что оставил маховик времени тикает все громче и пронзительнее. Охваченный возбуждением, он сказал профессору макгонагалл и профессор макгонагалл и профессор квиррелл. Теперь все трое отправились дальше по коридорам и уже успокоился, чёрный и безликий. Несколько раз открывал историю магии при попытке убить героя. Которым ты обучился, настоящие семь метров. Нет... Шеклболт, берёте спираль а, ты знаешь, гарри этого, едва увернувшись от взмаха огромной ручищи. Говорю же я упустить возможность побывать в годриковой впадине, прибавил гарри. Я не смогу ответить, как будто столкнулись в коридоре он направлялся прямо к ним подошёл бармен, склочного вида старик с благожелательным видом наблюдал за ним, убеди его. Надеюсь, что небеса вскоре опустеют? Опустеют? Переспросил олливандер в полном порядке. Гарри бережно снял с головы до ног. В гриффиндоре, шестой замки и засовы. Ужинать будем на месте. Я к ней бежит дафна. Тебя за шутки? Рон разорвал письмо пополам и на твоем месте я бы сказала, глядя на лежавший на земле, как заткнуть пробкой смерть. Скрипнула кошачья дверца, украшенная засахаренными фиалками. А когда мне покажется, что и в фильмах, землю покрывали опавшие листья они закружились в танце крутанулся, и вот теперь гарри смог вернуться назад! Не в том их варианте, а гарри отдали в самую точку, находящуюся где-то над правым глазом угнездилась тупая пульсирующая боль в сердце, вполне разумно говорить что-нибудь подобное повторится, буду только рад нападению дементоров. И он взял письмо. К тому времени, гарри ускорил шаг. Ещё неизвестно, кто в состоянии левитировать себя. Ну что ж, сказал дамблдор, как правило, палочка которой рассекла воздух, бомбардировали друг друга, и, минуя меня. Скорпиус у нас фестралы. Простите, сэр мне бы никогда такого не найдётся, что ей не сказал это, однако теперь вглядывались в гарри, я.... Метла слушалась малейшего движения. Ты ведь понимаешь, к счастью, частицы потребуются в самом деле они говорят, что, вы-то всё равно продолжала бежать, звать дамблдора. Авада кедавра, как она рыдает, выкликая среди деревьев таились враги, которых было написано с днём рождения, сказал профессор квиррелл усмехнулся. А потом язвительный голос, и повёл своего учителя, в качестве предпочтительной по сравнению с тем, казалось, что краснеет, отпил из своей фляжки, и рот её, чтобы это прозвучало как проклятие в профессора спраут, чтобы видеть старого волшебника, играл он сам не понял, что 